In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
driver.get("https://www.oddsportal.com/value-bets/#1/0/overall")

In [8]:
# Find all elements with the class 'visible'
value_bet_list = driver.find_elements(By.XPATH, "//div[@class='visible']")

In [19]:
# Parcourir chaque élément et extraire les données
for valuebet in value_bet_list:
    data = {}

    # 1. Extraire le chemin de navigation (Football / Thailand / Thai League 1)
    breadcrumb = valuebet.find_elements(By.CSS_SELECTOR, ".bg-gray-med_light a")
    data["breadcrumb"] = " / ".join([item.text for item in breadcrumb])

    # 2. Extraire les informations principales (O/U, date, heure, équipes)
    try:
        main_info = valuebet.find_element(By.CLASS_NAME, "min-mm\\:flex-2")
        data["main_info"] = {
            "match_type": main_info.find_element(By.CLASS_NAME, "bg-gray-light").text,
            "date": main_info.find_element(By.CSS_SELECTOR, ".text-gray-dark:nth-child(1)").text,
            "time": main_info.find_element(By.CSS_SELECTOR, ".text-gray-dark:nth-child(2)").text,
            "team_1": main_info.find_element(By.CSS_SELECTOR, ".participant-name:nth-child(1)").text,
            "team_2": main_info.find_element(By.CSS_SELECTOR, ".participant-name:nth-child(2)").text,
        }
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations principales : {e}")

    # 3. Extraire les données des paris (Outcome, Bookmaker, Odds, Value, Prob.)
    try:
        betting_table = valuebet.find_element(By.CSS_SELECTOR, ".min-mm\\:!overflow-visible ul")
        rows = betting_table.find_elements(By.CSS_SELECTOR, "li.flex.h-full")
        betting_data = []

        for row in rows:
            cells = row.find_elements(By.CSS_SELECTOR, "div.border-black-borders")
            betting_data.append({
                "outcome": cells[0].text if len(cells) > 0 else "",
                "bookmaker": cells[1].get_attribute("title") if len(cells) > 1 else "",
                "odds": cells[2].text if len(cells) > 2 else "",
                "value": cells[3].text if len(cells) > 3 else "",
                "probability": cells[4].text if len(cells) > 4 else "",
            })

        data["betting_data"] = betting_data
    except Exception as e:
        print(f"Erreur lors de l'extraction des données des paris : {e}")

    # Afficher les données extraites pour cet élément
    print(data)



Erreur lors de l'extraction des informations principales : Message: no such element: Unable to locate element: {"method":"css selector","selector":".participant-name:nth-child(2)"}
  (Session info: chrome=135.0.7049.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A508EFA5+77893]
	GetHandleVerifier [0x00007FF7A508F000+77984]
	(No symbol) [0x00007FF7A4E591BA]
	(No symbol) [0x00007FF7A4EAF16D]
	(No symbol) [0x00007FF7A4EAF41C]
	(No symbol) [0x00007FF7A4EA1B1C]
	(No symbol) [0x00007FF7A4ED716F]
	(No symbol) [0x00007FF7A4EA19E6]
	(No symbol) [0x00007FF7A4ED7340]
	(No symbol) [0x00007FF7A4EFF07F]
	(No symbol) [0x00007FF7A4ED6F03]
	(No symbol) [0x00007FF7A4EA0328]
	(No symbol) [0x00007FF7A4EA1093]
	GetHandleVerifier [0x00007FF7A5347B6D+2931725]
	GetHandleVerifier [0x00007FF7A5342132+2908626]
	GetHandleVerifier [0x00007FF7A53600F3+3031443]
	GetHandleVe

In [ ]:
# Fermer le driver
# driver.quit()

In [20]:
import pandas as pd

df = pd.DataFrame([data])
df.to_csv("oddsportal_data.csv", index=False, encoding='utf-8-sig')